### NMODL matexp solver

This notebook describes the implementation of the `matexp` solver, which solves the systems of ODEs defined in `KINETIC` blocks when the ODEs are *linear* and *coupled*.

For a more general tutorial on using the NMODL python interface, please see the [tutorial notebook](nmodl-python-tutorial.ipynb).

***

#### Mathematics
ODEs are linear if they can be written in the form $\frac{\partial x}{\partial t} = A \cdot x$ where $x$ is the state vector of the model and $A$ is the Jacobian matrix. The analytic solution to this initial value problem is $x = e^{A \Delta t} \cdot x_0$. The matrix $e^{A \Delta t}$ is known as the *propagator* matrix and it advances the state of the model by one time step. Since this is the exact solution, it is compatible with the Crank−Nicholson method which requires at least second-order accuracy.

***

#### Implementation
The `MatexpVisitor` implements the solver method `matexp`.

The visitor does the following:
* visit SOLVE statements to
    * determines which KINETIC blocks to solve
    * replace the SOLVE statement with a call to the solved PROCEDURE block
* visit KINETIC blocks to generate solved PROCEDURE blocks
    * first copy the KINETIC block into a new PROCEDURE block
        * append `_matexp` to the name of the new block
        * append `_steadystate` for steadystate solution
    * define the Jacobian matrix using the Eigen library by inserting a VERBATIM statement at the start of the block
    * replace reaction statements with equivalent assignments to the Jacobian matrix
        * For example `X <-> Y (A, B)` would become:
            * `J[X, X] -= A`
            * `J[Y, X] += A`
            * `J[Y, Y] -= B`
            * `J[X, Y] += B`
    * append the solution to the end of the block in a VERBATIM statement to
        * compute the propagator matrix and use it to advance the state of the model
        * conserve the sum of the states, if applicable
            * scale each state variable by the expected sum divided by the actual sum

***

#### Matrix Exponential Function
The exponential function is defined over matrices as well as scalars by its Taylor series expansion $e^x = \sum_{n=0}^{\infty} \frac{x^n}{n!}$. However, computing the exponential function by evaluating its Taylor series is slow and numerically unstable. There are many faster and more accurate methods of calculating the matrix exponential function. The NEURON simulator uses the [Eigen](https://eigen.tuxfamily.org) library for its advanced linear algebra functions, including for the matrix exponential function. For this task, the Eigen library implements the algorithm introduced in the publication:

> "The scaling and squaring method for the matrix exponential revisited," \
> By Nicholas J. Higham, \
> SIAM J. Matrix Anal. Applic., 26:1179–1193, 2005. \
> https://doi.org/10.1137/04061101X

***

#### Tests Cases
The unit tests may be helpful to understand what these functions are doing
  - `MatexpVisitor` tests are located in [test/nmodl/transpiler/unit/visitor/matexp.cpp](https://github.com/neuronsimulator/nrn/blob/master/test/nmodl/transpiler/unit/visitor/matexp.cpp), and tests involving `matexp` have the tag "`[matexp]`"

***

#### Examples

In [1]:
%%capture
! pip install neuron

In [11]:
import neuron.nmodl.dsl as nmodl


def run_matexp_solver(mod_string):
    # Parse NMDOL file (supplied as a string) into AST
    driver = nmodl.NmodlDriver()
    AST = driver.parse_string(mod_string)

    # Run SymtabVisitor to generate Symbol Table
    nmodl.symtab.SymtabVisitor().visit_program(AST)

    # Run matexp solver
    nmodl.visitor.MatexpVisitor().visit_program(AST)

    # Return the solution as a string
    return nmodl.to_nmodl(AST)

##### Ex. 1
Linear kinetic model with conserved sum of states

In [16]:
ex1 = """
STATE {
    A
    B
}
BREAKPOINT {
    SOLVE states METHOD matexp
}
KINETIC states {
    ~ A <-> B (0.123, 0.456)
    CONSERVE A + B = 0.789
}
"""
print(run_matexp_solver(ex1))

VERBATIM
#undef g 
 #undef F 
 #undef t 
 #include <unsupported/Eigen/MatrixFunctions>
ENDVERBATIM

STATE {
    A
    B
}

BREAKPOINT {
    SOLVE states_matexp
}

PROCEDURE states_matexp() (){
    VERBATIM
Eigen::Matrix<double, 2, 2> nmodl_eigen_jm = Eigen::Matrix<double, 2, 2>::Zero();
double* nmodl_eigen_j = nmodl_eigen_jm.data();
ENDVERBATIM
    nmodl_eigen_j[0] = nmodl_eigen_j[0]-(0.123)*dt
    nmodl_eigen_j[1] = nmodl_eigen_j[1]+(0.123)*dt
    nmodl_eigen_j[3] = nmodl_eigen_j[3]-(0.456)*dt
    nmodl_eigen_j[2] = nmodl_eigen_j[2]+(0.456)*dt
    VERBATIM
Eigen::Matrix<double, 2, 1> nmodl_eigen_xm;
double* nmodl_eigen_x = nmodl_eigen_xm.data();
ENDVERBATIM
    nmodl_eigen_x[0] = A
    nmodl_eigen_x[1] = B
    VERBATIM
Eigen::Matrix<double, 2, 1> nmodl_eigen_ym = nmodl_eigen_jm.exp() * nmodl_eigen_xm;
nmodl_eigen_ym *= (0.789) / nmodl_eigen_ym.sum();
double* nmodl_eigen_y = nmodl_eigen_ym.data();
ENDVERBATIM
    A = nmodl_eigen_y[0]
    B = nmodl_eigen_y[1]
}



##### Ex. 2
Non-linear ODEs

The matexp solver checks for linear equations and raises exception if it finds any non-linearities.
It asserts that each reaction has exactly one reactant and one product.
This means that the following statements will cause errors:

```
KINETIC nonlinear_errors {
    ~ A <-> B + C (kf, kr) : Multiple products!
    ~ 2 A <-> C (kf, kr)   : Multiple reactants!
    ~ A << (kf)            : Zero reactants!
}
```

However the solver does not check that the kinetic rates are independent from state variables, and so the following non-linear equation will be accepted by the program and solved inaccurately.

```
KINETIC nonlinear_silent_bug {
    ~ A <-> B (func(A), kr)
}
```


##### Ex. 3
Unsupported features

The reaction operator `->` is not supported and will raise an exception. This operator is a shorthand way of specifing reaction that have zero products. For example the statement `A -> (rate)` is eqivalent to `A <-> (rate, 0)` which specifies that the state variable `A` decays exponentially. Although technically linear, the matexp solver does not support exponential decay.

```
KINETIC unsupported_features {
    ~ A -> (k_decay)
}
```
